# Learn Posture

use machine learning to recognize robot's posture (following the example in [scikit-learn-intro.ipynb](./scikit-learn-intro.ipynb) )

## 1. Data collection

We have colleceted data before, you need to add new data if you want to add new posture.

* the dateset are in *robot_pose_data* folder
* each file contains the data belongs to this posture, e.g. the data in *Back* file are collected when robot was in "Back" posture
* the data file can be load by ```pickle```, e.g. ```pickle.load(open('Back'))```, the data is a list of feature data
* the features (e.g. each row of the data) are ['LHipYawPitch', 'LHipRoll', 'LHipPitch', 'LKneePitch', 'RHipYawPitch', 'RHipRoll', 'RHipPitch', 'RKneePitch', 'AngleX', 'AngleY'], where 'AngleX' and 'AngleY' are body angle (e.g. ```Perception.imu```) and others are joint angles.

## 2. Data preprocessing

In [1]:
%pylab inline
import pickle
from os import listdir, path
import numpy as np
from sklearn import svm, metrics

ROBOT_POSE_DATA_DIR = 'robot_pose_data'

Populating the interactive namespace from numpy and matplotlib


In [2]:
classes = listdir(ROBOT_POSE_DATA_DIR)
print classes

['Back', 'Belly', 'Crouch', 'Frog', 'HeadBack', 'Knee', 'Left', 'Right', 'Sit', 'Stand', 'StandInit']


In [3]:
def load_pose_data(i):
    '''load pose data from file'''
    data = []
    target = []
    # YOUR CODE HERE
    # Data extraction already done?
    filename = path.join(ROBOT_POSE_DATA_DIR, classes[i])
    data = pickle.load(open(filename))
    target = [i] * len(data)
    return data, target

print len(load_pose_data(2)[0]), len(load_pose_data(2)[1])

30 30


In [10]:
# load all the data
all_data = []
all_target = []
# YOUR CODE HERE
for i in range(len(classes)):
    class_data, class_target = load_pose_data(i)
    for data_point, target_point in zip(class_data, class_target):
        all_data.append(data_point)
        all_target.append(target_point)

print 'total number of data', len(all_data)

total number of data 222


In [47]:
# shuffule data
training_size = 0.7
permutation = np.random.permutation(len(all_data))
n_training_data = int(len(all_data) * training_size)

training_data = list(all_data[i] for i in permutation[:n_training_data])
testing_data = list(all_data[i] for i in permutation[n_training_data:])

training_target = list(all_target[i] for i in permutation[:n_training_data])
testing_target = list(all_target[i] for i in permutation[n_training_data:])

print len(training_data), len(testing_data)

155 67


## 3. Learn on training data

In scikit-learn, an estimator for classification is a Python object that implements the methods fit(X, y) and predict(T). An example of an estimator is the class sklearn.svm.SVC that implements support vector classification.

In [48]:
clf = svm.SVC(gamma=0.001, C=100.)

### learning

In [49]:
# YOUR CODE HERE
clf.fit(training_data, training_target)

SVC(C=100.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.001, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

### predicting

In [41]:
print clf.predict(all_data[-1:]), all_target[-1:]

[10] [10]


## 4. Evaluate on the test data

In [36]:
def evaluate(expected, predicted):
    print("Classification report:\n%s\n" % metrics.classification_report(expected, predicted))

    print("Confusion matrix:\n%s" % metrics.confusion_matrix(expected, predicted))

In [50]:
expected = []
predicted = []
# YOUR CODE HERE
expected = testing_target
predicted = clf.predict(testing_data)

evaluate(expected, predicted)

Classification report:
             precision    recall  f1-score   support

          0       1.00      1.00      1.00         5
          1       1.00      1.00      1.00         5
          2       0.89      1.00      0.94         8
          3       1.00      1.00      1.00         2
          4       1.00      1.00      1.00         5
          5       1.00      1.00      1.00         3
          6       1.00      1.00      1.00         6
          7       1.00      0.83      0.91         6
          8       1.00      1.00      1.00         9
          9       1.00      1.00      1.00         3
         10       1.00      1.00      1.00        15

avg / total       0.99      0.99      0.98        67


Confusion matrix:
[[ 5  0  0  0  0  0  0  0  0  0  0]
 [ 0  5  0  0  0  0  0  0  0  0  0]
 [ 0  0  8  0  0  0  0  0  0  0  0]
 [ 0  0  0  2  0  0  0  0  0  0  0]
 [ 0  0  0  0  5  0  0  0  0  0  0]
 [ 0  0  0  0  0  3  0  0  0  0  0]
 [ 0  0  0  0  0  0  6  0  0  0  0]
 [ 0  0  1  0 

In [71]:
# loop with different random permutation to see if on average good or not
training_size = 0.7
number_of_runs = 500 # Put this number lower if slow pc
errors = []

for i in range(number_of_runs):
    permutation = np.random.permutation(len(all_data))
    n_training_data = int(len(all_data) * training_size)

    training_data = list(all_data[i] for i in permutation[:n_training_data])
    testing_data = list(all_data[i] for i in permutation[n_training_data:])

    training_target = list(all_target[i] for i in permutation[:n_training_data])
    testing_target = list(all_target[i] for i in permutation[n_training_data:])

    clf.fit(training_data, training_target)
    
    expected = testing_target
    predicted = clf.predict(testing_data)
    error_rate = float(sum([1 for x in (expected - predicted) if x!=0]))/len(all_data)
    
    errors.append(error_rate)
    
average_error = float(sum(errors))/len(errors)
print "Average accuracy is: ", (1 - average_error)

Average accuracy is:  0.997351351351


It seems the accuracy is at about 99.7%

## 5. Deploy to the real system

We can simple use `pickle` module to serialize the trained classifier.

In [67]:
import pickle
ROBOT_POSE_CLF = 'robot_pose.pkl'
pickle.dump(clf, open(ROBOT_POSE_CLF, 'w'))

Then, in the application we can load the trained classifier again.

In [69]:
clf2 = pickle.load(open(ROBOT_POSE_CLF))
clf2.predict(all_data[-1:]), all_target[-1:]

(array([10]), [10])